In [2]:
# pip install joblib
# pip install nibabel 
# !pip install matplotlib
# !pwd

# !pip install scikit-image
# !pip install plotly


import os
import nibabel as nib
import numpy as np

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".nii") or filename.endswith(".nii.gz"): # Check for specific image file extensions
            img_path = os.path.join(folder_path, filename)
            img = nib.load(img_path)
            img_data = img.get_fdata()
            print(f"Shape of image {filename}: {img_data.shape}")
            images.append(img_data)
    return np.array(images)

# Example usage:
path = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'
folder_path = path # Replace with the path to your folder
loaded_images = load_images_from_folder(folder_path)

Shape of image sub-A00000541_ses-20100101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000456_ses-20090101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000838_ses-20100101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000368_ses-20110101_acq-mprage_run-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000909_ses-20110101_acq-mprage_run-02_T1w.nii: (192, 256, 256)
Shape of image sub-A00000865_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)
Shape of image sub-A00000844_ses-20100101_acq-mprage_echo-01_T1w.nii: (192, 256, 256)


In [3]:
for i in loaded_images:
    print(i.shape)    

(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)
(192, 256, 256)


In [2]:
len(loaded_images)

/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again

7

In [1]:
!pip install tqdm

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/home/caitlyn/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/home/caitlyn/.local/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/caitlyn/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/home/caitlyn/.local/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _

In [2]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed




# Running FSL ANAT and getting FSL ANAT Folders 

# n_jobs  = -1 # Number of jobs to run in parallel. -1 means use all available processors.
n_jobs = 10

def process_anat_volumes(directory, n_jobs=6):
    """
    Process anatomical volumes for files in a specified directory using parallel computing.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    process_anat_volumes('/path/to/your/directory/', n_jobs=8)
    """
    def anat_volumes(filename):
        full_path = os.path.join(directory, filename)
        return subprocess.run(["fsl_anat", "-i", full_path], capture_output=True)

    Parallel(n_jobs=n_jobs)(delayed(anat_volumes)(filename) for filename in os.listdir(directory))

process_anat_volumes(directory, n_jobs=n_jobs)




In [ ]:

# upon running the above which is a function, it will return a subprocess.CompletedProcess object containing information about the completed process.
# a folder with various files will be created in the directory where the function is run.  This folder will be named after the file that was processed and will contain the following files:
#  the naming will be example_patient.nii.gz --> example_patient.anat



In [3]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed

directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'

def invwarp(directory, ref_file, warp_file, out_file_name, verbose=False):
    """
    Run the FSL command 'invwarp' to invert a non-linear warp field.

    Parameters:
    directory (str): The directory path where the files are located.
    ref_file (str): The reference file in the specified directory.
    warp_file (str): The warp file in the specified directory.
    out_file_name (str): The name of the output file.
    verbose (bool): If True, display verbose output (default is False).

    Returns:
    CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

    Example:
    invwarp('/path/to/your/directory/', 'T1.nii.gz', 'T1_to_MNI_nonlin_field.nii.gz', 'std_subject_space', verbose=True)
    """
    ref_path = f"{directory}/{ref_file}"
    warp_path = f"{directory}/{warp_file}"
    out_path = f"{directory}/Warps/{out_file_name}"
    verbose_arg = "--verbose" if verbose else ""

    return subprocess.run(["invwarp", f"--ref={ref_path}", f"--warp={warp_path}", f"--out={out_path}", verbose_arg], capture_output=True)


# For the Cortical Regions- pulling from FSL_Anat Folder 



def applywarp_cort(directory, ref_file, in_file, warp_file, out_file_name, interp_method='nn'):
    """
    Apply a warp to an input file and resample it to the space of a reference file.

    Parameters:
    directory (str): The directory path where the files are located.
    ref_file (str): The reference file in the specified directory.
    in_file (str): The input file to be warped.
    warp_file (str): The warp file in the specified directory.
    out_file_name (str): The name of the output file.
    interp_method (str): The interpolation method to be used (default is 'nn').

    Returns:
    CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

    Example:
    applywarp_cort('/path/to/your/directory/', 'T1.nii.gz', 'HarvardOxford-cort-maxprob-thr50-2mm.nii.gz', 'std_subject_space.nii.gz', 'HO_in_subj_t1_space.nii.gz', interp_method='trilinear')
    """
    ref_path = f"{directory}/{ref_file}"
    in_path = f"/usr/local/fsl/data/atlases/HarvardOxford/{in_file}"
    warp_path = f"{directory}/Warps/{warp_file}"
    out_path = f"{directory}/Warps/{out_file_name}"

    return subprocess.run(["applywarp", f"--ref={ref_path}", f"--in={in_path}", f"--warp={warp_path}", f"--out={out_path}", f"--interp={interp_method}"], capture_output=True)



def applywarp_subcort(directory, ref_file, in_file, warp_file, out_file_name, interp_method='nn'):
    """
    Apply a warp to an input file and resample it to the space of a reference file.

    Parameters:
    directory (str): The directory path where the files are located.
    ref_file (str): The reference file in the specified directory.
    in_file (str): The input file to be warped.
    warp_file (str): The warp file in the specified directory.
    out_file_name (str): The name of the output file.
    interp_method (str): The interpolation method to be used (default is 'nn').

    Returns:
    CompletedProcess: A subprocess.CompletedProcess object containing information about the completed process.

    Example:
    applywarp_subcort('/path/to/your/directory/', 'T1.nii.gz', 'HarvardOxford-sub-maxprob-thr50-2mm.nii.gz', 'std_subject_space.nii.gz', 'subcort_HO_in_subj_t1_space.nii.gz', interp_method='trilinear')
    """
    ref_path = f"{directory}/{ref_file}"
    in_path = f"/usr/local/fsl/data/atlases/HarvardOxford/{in_file}"
    warp_path = f"{directory}/Warps/{warp_file}"
    out_path = f"{directory}/Warps/{out_file_name}"

    return subprocess.run(["applywarp", f"--ref={ref_path}", f"--in={in_path}", f"--warp={warp_path}", f"--out={out_path}", f"--interp={interp_method}"], capture_output=True)


def apply_warps(path):
    """
    Apply warps to specified files and save the results in the 'Warps' folder.

    Parameters:
    path (str): The directory path where the files are located.

    Returns:
    None

    Example:
    apply_warps('/path/to/your/directory/')
    """
    folder = os.path.join(path, "Warps") 
    if not os.path.exists(folder):
        os.mkdir(folder)

    invwarp(path)
    time.sleep(3)  # Sleep for 3 seconds to ensure completion before the next step
    applywarp_cort(path)
    applywarp_subcort(path)



def apply_warps_parallel(directory, n_jobs=6):
    """
    Apply warps to files in a specified directory in parallel.

    Parameters:
    directory (str): The directory path where the files are located.
    n_jobs (int): The number of jobs to run in parallel (default is 6).

    Returns:
    None

    Example:
    apply_warps_parallel('/path/to/your/directory/', n_jobs=8)
    """
    def apply_warps(path):
        folder = os.path.join(path, "Warps") 
        if not os.path.exists(folder):
            os.mkdir(folder)

        invwarp(path)
        time.sleep(3)  # Sleep for 3 seconds to ensure completion before the next step
        applywarp_cort(path)
        applywarp_subcort(path)

    Parallel(n_jobs=n_jobs)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))



apply_warps_parallel(directory, n_jobs=15)
#questions to be asked

# How long does this take for each image? 
# How long does this take for all images?
    
# What folders does this pull from?
# Do i need to move folders? 

# What is the output?


# What function do I need to run to get a total output? 


TypeError: invwarp() missing 3 required positional arguments: 'ref_file', 'warp_file', and 'out_file_name'

In [2]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed



directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'  #to practice on how pipeline will run with multiple images 

def invwarp(directory):
    return subprocess.run(["invwarp", "--ref="+directory+"/T1.nii.gz", "--warp="+directory+"/T1_to_MNI_nonlin_field.nii.gz", "--out="+directory+"/Warps/"+"std_subject_space", "--verbose"], capture_output=True)


def applywarp_cort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-cort-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)

def applywarp_subcort(directory):
    return subprocess.run(["applywarp","--ref="+directory+"/T1.nii.gz", "--in=/usr/local/fsl/data/atlases/HarvardOxford/HarvardOxford-sub-maxprob-thr50-2mm.nii.gz", 
               "--warp="+directory+"/Warps/"+"std_subject_space.nii.gz", 
               "--out="+directory+"/Warps/"+"subcort_HO_in_subj_t1_space.nii.gz",  "--interp=nn"], capture_output=True)


def apply_warps(path):
    
    folder = os.path.join(path, "Warps") 
    if not os.path.exists(folder):
        os.mkdir(folder)

    invwarp(path)
    print(path + ' invwarp done')
    time.sleep(3)
    applywarp_cort(path)
    print(path + ' cort done')
    applywarp_subcort(path)
    print(path + ' subcort done')

    from joblib import Parallel, delayed
Parallel(n_jobs=10)(delayed(apply_warps)(file) for file in glob.glob(os.path.join(directory, '*.anat')))




[None, None, None, None, None, None, None]

In [ ]:
# fslstats -K subcort_HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the subcortical regions
# fslstats -K HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the cortical regions


def get_vols(vol_files, label):
    
    patient = {}
    
    for file in vol_files:
        
        path = os.path.basename(file)
        
        label_vals = [10, 11, 12, 13, 16, 17, 18, 26, 49, 50, 51, 52, 53, 54, 58]
        length = len(label_vals)

        vols = {}
        
        
        for i in label_vals:
            out = sout = subprocess.Popen(["fslstats", file, "-l", str((i+0.5)-1), "-u", str(i+0.5), "-V"], 
                   stdout=subprocess.PIPE, 
                   stderr=subprocess.STDOUT)
            stdout,stderr = out.communicate()
            vols["LABEL"] = label
            vols[str(i)] = [float(str(stdout.split()[0])[2:-1]), float(str(stdout.split()[1])[2:-1])]

        patient[path[:-29]] = vols
    
    return patient

In [ ]:
# fslstats -K subcort_HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the subcortical regions
# fslstats -K HO_in_subj_t1_space.nii.gz T1.nii.gz -M -V  # for the cortical regions

# I need to
# 1. feed in the parent directory
# 2. feed in the label for each file and where the file is located
# 3. apply fslstats for cort and applywarp_subcort
# 4. collect the outputs and feed into dictionary
    



def getVol_Dataframe(directory):

    brain_volumes = pd.DataFrame()

    for file in glob.glob(os.path.join(directory)):

        path = os.path.basename(file)

        print(file)

        # vols = {}

        # for i in label_vals:
        #     print(i)
        #     out_cort = sout = subprocess.Popen(["fslstats", "-K", "./Warps/HO_in_subj_t1_space.nii.gz", "T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        #     stdout,stderr = out.communicate()
        #     out_subcort = sout = subprocess.Popen(["fslstats", "-K", "./Warps/subcort_HO_in_subj_t1_space.nii.gz", "T1.nii.gz", "-V"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        #     stdout,stderr = out.communicate()

        #     vols["LABEL"] = label
        #     vols[str(i)] = [float(str(stdout.split()[0])[2:-1]), float(str(stdout.split()[1])[2:-1])]

        # patient[path[:-29]] = vols

In [4]:
import subprocess
import os
import time
import glob
from joblib import Parallel, delayed



directory = '/mnt/md0/cads-phd/explainbrainROI/spr_mini_try_again/'  #to practice on how pipeline will run with multiple images 

for file in glob.glob(os.path.join(directory)):
    
    path = os.path.basename(file)

    print(path)